<a href="https://colab.research.google.com/github/madhupathy/PGP-GABA-UTA-JUL25/blob/main/Medical_Assistant_RAG_FullCode_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Assistant (RAG) — Full‑Code Starter

**Goal:** Answer clinical questions using a Retrieval‑Augmented Generation (RAG) pipeline over the Merck Manual PDF. This notebook covers installs, GPU check, data ingestion, embeddings + FAISS, prompt‑sweeps, RAG QA, and evaluation.

**Submission:** Export to HTML at the end.

> ⚠️ Educational use only. Not a substitute for professional medical advice.


## 0) Runtime & Setup (Colab)
- *Runtime → Change runtime type → GPU (T4 recommended)*
- If CUDA errors occur, re‑select GPU and **Restart runtime**.

In [13]:
#@title GPU check
!nvidia-smi || echo "No GPU visible — enable GPU in Runtime > Change runtime type"


/bin/bash: line 1: nvidia-smi: command not found
No GPU visible — enable GPU in Runtime > Change runtime type


## 1) Installs

In [14]:
#@title Install libraries (may take a few minutes)
!pip -q install --upgrade pip
!pip -q install faiss-cpu sentence-transformers pypdf langchain langchain-community langchain-text-splitters
!pip -q install llama-cpp-python==0.2.90  # works well on Colab T4; adjust if needed
# Optional: reranker
!pip -q install FlagEmbedding                    # BGE reranker


## 2) Configuration

In [20]:
#@title Configuration
from pathlib import Path
PDF_PATH = Path('/content/medical_diagnosis_manual.pdf')
EMBED_MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
FAISS_DIR = Path('/content/faiss_index')
FAISS_DIR.mkdir(exist_ok=True, parents=True)

# LLM config (llama-cpp); alternatively, plug in OpenAI etc.
LLAMA_MODEL_PATH = '/content/llama-3.2-3b-instruct.Q4_K_M.gguf'
TEMPERATURE = 0.2
MAX_TOKENS = 1024
TOP_P = 0.95
TOP_K = 40

!pip -q install -U huggingface_hub
from huggingface_hub import hf_hub_download

MODEL_REPO = "bartowski/Llama-3.2-3B-Instruct-GGUF"
FILENAME   = "Llama-3.2-3B-Instruct-Q4_K_M.gguf"

local_path = hf_hub_download(repo_id=MODEL_REPO, filename=FILENAME, local_dir="/content", local_dir_use_symlinks=False)
print("Saved to:", local_path)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

Saved to: /content/Llama-3.2-3B-Instruct-Q4_K_M.gguf


## 3) Load & Chunk PDF

In [16]:
#@title Load PDF and chunk
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

assert PDF_PATH.exists(), f"Upload your PDF to {PDF_PATH}"
loader = PyPDFLoader(str(PDF_PATH))
docs = loader.load()
print(f"Pages loaded: {len(docs)}")

splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)
chunks = splitter.split_documents(docs)
print(f"Chunks: {len(chunks)} (size≈1200, overlap=150)")


Pages loaded: 4114
Chunks: 14561 (size≈1200, overlap=150)


## 4) Embeddings & FAISS

In [17]:
#@title Build / Load FAISS index
from sentence_transformers import SentenceTransformer
import faiss, numpy as np

embedder = SentenceTransformer(EMBED_MODEL_NAME)
embs = embedder.encode([c.page_content for c in chunks], show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
index = faiss.IndexFlatIP(embs.shape[1])
index.add(embs)
print('FAISS index size:', index.ntotal)

# Store minimal store (texts + metadata)
texts = [c.page_content for c in chunks]
metas = [c.metadata for c in chunks]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/456 [00:00<?, ?it/s]

FAISS index size: 14561


## 5) Retriever

In [18]:
#@title Define retriever (similarity search)
import numpy as np

def retrieve(query, k=6):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q, k)
    ctxs = [(texts[i], metas[i]) for i in I[0]]
    return ctxs

# quick test
print(len(retrieve("sepsis hour-1 bundle", k=3)))


3


## 6) LLM (llama-cpp) — local

In [22]:
#@title Load llama-cpp (provide GGUF path above)
from llama_cpp import Llama
llm = Llama(model_path=LLAMA_MODEL_PATH, n_gpu_layers=50, n_ctx=4096, n_threads=4, verbose=False)



## 7) Prompt Templates

In [23]:
#@title Prompts (baseline, engineered, RAG)
BASELINE_SYS = """You are a careful clinical assistant. If unsure, say so and ask for clarification. Do not fabricate.
Educational use only — not medical advice.
"""

ENGINEERED_SYS = """You are an ICU attending. Answer concisely in bullets and include a short ‘Rationale’. If evidence is insufficient, state that explicitly.
Return JSON with fields: answer, rationale, red_flags.
"""

RAG_SYS = """You are a clinical assistant. Answer ONLY using the provided CONTEXT.
- If the context is insufficient, say "Insufficient context" and request what is missing.
- Cite chunk IDs like [C1], [C2] from the given CONTEXT.
Educational use only — not medical advice.
"""

USER_QS = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are treatments and possible causes for sudden patchy hair loss (localized bald spots on scalp)?",
    "What treatments are recommended for a person with brain tissue injury (TBI) causing temporary or permanent impairment?"
]


## 8) Generation utilities

In [24]:
#@title Generation helpers
from textwrap import shorten

def chat(llm, system, user, temperature=0.2, top_p=0.95, max_tokens=1024, top_k=40):
    prompt = f"<s>[INST]<<SYS>>\n{system}\n<</SYS>>\n{user}[/INST]"
    out = llm(prompt, temperature=temperature, top_p=top_p, max_tokens=max_tokens, top_k=top_k)
    return out["choices"][0]["text"].strip()

# RAG
def answer_with_rag(question, k=6, temperature=0.2):
    ctxs = retrieve(question, k=k)
    ctx_text = "\n\n".join([f"[C{i+1}] {shorten(t[:1200], width=1200)}" for i,(t,_) in enumerate(ctxs)])
    user = f"CONTEXT:\n{ctx_text}\n\nQUESTION: {question}\n\nAnswer with citations [C#]."
    return chat(llm, RAG_SYS, user, temperature=temperature)


## 9) Baseline & Prompt‑Engineered QA (5+ variants)

In [ ]:
#@title Run baselines & prompt sweeps
import itertools

param_grid = {
  'temperature': [0.0, 0.2],
  'top_p': [0.9, 1.0],
  'top_k': [20, 40]
}

runs = []
for q in USER_QS:
    # Baseline
    out = chat(llm, BASELINE_SYS, q, temperature=0.2)
    runs.append({"mode":"baseline","q":q,"params":{"temperature":0.2},"answer":out})

    # Prompt-engineered sweeps (at least 5 combos)
    combos = list(itertools.product(param_grid['temperature'], param_grid['top_p'], param_grid['top_k']))[:5]
    for t,tp,tk in combos:
        out = chat(llm, ENGINEERED_SYS, q, temperature=t, top_p=tp, top_k=tk)
        runs.append({"mode":"engineered","q":q,"params":{"temperature":t,"top_p":tp,"top_k":tk},"answer":out})

len(runs)


## 10) RAG QA

In [ ]:
#@title Run RAG answers
for q in USER_QS:
    ans = answer_with_rag(q, k=6, temperature=0.2)
    runs.append({"mode":"rag","q":q,"params":{"k":6,"temperature":0.2},"answer":ans})

len(runs)


## 11) Evaluation — Groundedness, Relevance, Context Precision

In [ ]:
#@title Simple automatic rubric (embedding similarity)
import numpy as np

def cosine(a,b):
    return float(np.dot(a,b) / (np.linalg.norm(a)*np.linalg.norm(b) + 1e-8))

# Groundedness: max similarity between answer and its retrieved context chunks (proxy)
# Relevance: similarity between answer and question
# Context precision: average similarity of retrieved chunks to the question (for RAG)

def embed_text(t: str):
    return embedder.encode([t], convert_to_numpy=True, normalize_embeddings=True)[0]

question_vecs = {q: embed_text(q) for q in USER_QS}

evals = []
for r in runs:
    q = r['q']
    ans = r['answer']
    ans_vec = embed_text(ans)
    rel = cosine(ans_vec, question_vecs[q])
    grounded = None
    ctx_prec = None
    if r['mode'] == 'rag':
        ctxs = retrieve(q, k=6)
        ctx_vecs = [embed_text(t) for t,_ in ctxs]
        grounded = max(cosine(ans_vec, v) for v in ctx_vecs)
        ctx_prec = float(np.mean([cosine(question_vecs[q], v) for v in ctx_vecs]))
    evals.append({**r, 'metrics': {'relevance': rel, 'groundedness': grounded, 'context_precision': ctx_prec}})

len(evals)


## 12) Display & Export

In [ ]:
#@title Tabulate results and export to JSON
import pandas as pd

df = pd.DataFrame([
    {
        'mode':e['mode'],
        'question': e['q'],
        'params': str(e['params']),
        'relevance': round(e['metrics']['relevance'],3),
        'groundedness': None if e['metrics']['groundedness'] is None else round(e['metrics']['groundedness'],3),
        'context_precision': None if e['metrics']['context_precision'] is None else round(e['metrics']['context_precision'],3),
        'answer': e['answer'][:400]
    } for e in evals
])

from caas_jupyter_tools import display_dataframe_to_user

display_dataframe_to_user('QA & Evaluation Summary', df)

df.to_json('/content/qa_eval_results.json', orient='records', indent=2)
print('Saved /content/qa_eval_results.json')


### HTML Export (for submission)

In [ ]:
#@title Save this notebook as HTML
!jupyter nbconvert --to html /content/Medical_Assistant_RAG_FullCode_Starter.ipynb --output /content/Medical_Assistant_RAG_Submission.html || echo 'Open: File > Download > .ipynb then convert locally if nbconvert is unavailable.'


## Notes & Citations
- Merck Manual (Professional Edition): Sepsis & Septic Shock; Appendicitis; Alopecia Areata; Traumatic Brain Injury.
- Surviving Sepsis Campaign (SCCM) Adult Guidelines (Hour‑1 Bundle), 2021.
